# TAP001 - Mobile Malware Kill Chain

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK

**Threat Actor Profile (TAP):** 001 <br/>
**Kill Chain**: Mobile Malware - Ransomware Script Variant

This notebook demonstrates the new threat actor profile (TAP) on the UC7 network infrastructure. In this scenario, a `some_tech` employee within the development project network plugs in his personal device. Whilst browsing, they encounter a ransomware virus which moves onto the host machine thus triggering the malware! 


This malware variant targets the database service's file directly, rather than the disrupting the service, hence why in this scenario the `data-service` is still functional after becoming corrupted.

This new red agent intends to introduce more realistic impacts to the observation space, such as files created/removed and applications installing mid-episode whilst still providing the configurability needed for effective blue agent training.

In [ ]:
!primaite setup

In [ ]:
# Importing the necessary PrimAITE libraries
from primaite.session.environment import PrimaiteGymEnv
import yaml
from primaite.game.agent.scripted_agents.TAP001 import TAP001, MobileMalwareKillChain
from primaite.config.load import load, _EXAMPLE_CFG
from deepdiff.diff import DeepDiff

In [ ]:
# Utility functions

def display_obs_diffs(old, new, step_counter):
    """
    Use DeepDiff to extract and display differences in old and new instances of
    the observation space.

    :param old: observation space instance.
    :param new: observation space instance.
    :param step_counter: current step counter.
    """
    print("\nObservation space differences")
    print("-----------------------------")
    diff = DeepDiff(old, new)
    print(f"Step {step_counter}")
    for d,v in diff.get('values_changed', {}).items():
        print(f"{d}: {v['old_value']} -> {v['new_value']}")



### **Notebook Sections:**
1. Notebook Intro
2. Notebook Setup
3. Initial States
4. Attack Trigger
5. Post Attack States
6. Configurations

### **Notebook Intro** | **TAP001 - Mobile Malware Kill Chain** 

The TAP001 kill chain is comprised of 6 different stages. <br> The table below describes the impact of the current TAP001 implementation as well as the attack stage order.

_DOWNLOAD_

|Index|Attack Stage|OBS Impact|Narrative|
|-----|------------|----------|---------|
|1|Download|**Starting Host** HOST:num_file_creations: HOST:FOLDER:FILES:* |The employee encounters a malicious payload through web browsing on his mobile phone, often disguised as legitimate content.|

_INSTALL_

|Index|Attack Stage|OBS Impact|Narrative|
|-----|------------|----------|---------|
|2|Install | **Starting Host** HOST:FOLDER:FILES:num_access:|The malware is activated, either by the employee opening the file or automatically upon download, initiating its malicious functions.|

_ACTIVATE_

|Index|Attack Stage|OBS Impact|Narrative|
|-----|------------|----------|---------|
|3|Activate | **Starting Host** HOST:APPLICATION:operating_status|The malware installs itself on the terminal, often seeking to gain persistence by embedding itself into system processes or startup routines.|

_PROPAGATE_

|Index|Attack Stage|OBS Impact|Narrative|
|-----|------------|----------|---------|
|4|Propagate| **Multiple Hosts** HOST:NICS:NIC:TRAFFIC:PROTOCOL:PORT:*|The malware attempts to spread to other systems or networks, looking for vulnerable services.|

_COMMAND_AND_CONTROL_

|Index|Attack Stage|OBS Impact|Narrative|
|-----|------------|----------|---------|
|5|Command and Control|**Starting Host** HOST:NICS:NIC:TRAFFIC:PROTOCOL:PORT:* HOST:APPLICATIONS:APPLICATION:*|The malware establishes a connection to an external command and control (C&C) server, receiving instructions and possibly exfiltrating data.|

_PAYLOAD_

|Index|Attack Stage|OBS Impact|Narrative|
|-----|------------|----------|---------|
|6A|Corruption| **Target Host** HOST:FOLDERS:FOLDER:FILES:FILE:num_access HOST:num_file_creations  HOST:num_file_deletions  HOST:FOLDERS:FOLDER:FILES:FILE:health_status | The attacker configures the ransomware script to target the IP address of the discovered database, then it performs its intended malicious activities, allowing it to corrupt the Database whilst leaving the database service operable|
|6B|Exfiltration| **Starting Host** HOST:FOLDER:FILES:FILE HOST:NICS:NIC:TRAFFIC:PROTOCOL:PORT:* HOST:APPLICATIONS:APPLICATION:* | The attacker remotely logins into the target host and exfiltrates the database.db file onto the starting host which then relays this file back to the C2 Server.|



## **Notebook Setup** | **Network Configuration:**

This notebook uses the same network setup as UC7. Please refer to the main [UC7-E2E-Demo notebook for further reference](./UC7-E2E-Demo.ipynb).

<p align="center">
    <a href="./_package_data/uc7/uc7_tap001/uc7_tap001_main.png" target="_blank">
        <img src="./_package_data/uc7/uc7_tap001/uc7_tap001_main.png" alt="Image" style="width:1000px;">
    </a>  
    
</p>
<p style="text-align: center;"> TAP001 Mobile Malware Kill Chain Demonstration - Ransomware Variant</p>

_(Click to enlarge)_

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config.yaml", mode="r") as uc7_config:
    cfg = yaml.safe_load(uc7_config)
    cfg["agents"][33]["agent_settings"]["flatten_obs"] = False
    cfg['io_settings']['save_sys_logs'] = True # Saving syslogs
    cfg['io_settings']['save_agent_actions'] = True # Saving attacker logs
env = PrimaiteGymEnv(env_config=cfg)
env.game.simulation.network.show()

### **Notebook Setup** | Instantiating Relevant Simulation Objects

Simulation objects can be instantiated and called independently of agents via the environment.game.simulation (PrimAITE API) which is useful for demonstrating the simulation state at different points within an episode.

Any readers unfamiliar with the UC7 scenario should refer to the [main UC7 notebook for further details](./UC7-E2E-Demo.ipynb)

In [ ]:
env.reset() # resetting the environment
# The TAP001 Agent
tap001 = env.game.agents.get("attacker")
# A potential starting client
starting_host = env.game.simulation.network.get_node_by_hostname('ST_PROJ-A-PRV-PC-1')

# The database server which acts as the initial target of the ransomware kill-chain
database_server = env.game.simulation.network.get_node_by_hostname('ST_DATA-PRV-SRV-DB')

## **Prior To Attack** | Initial States:

The first section of this notebook displays the relevant initial `observation_space` (OBS) of effected nodes before the attack takes place.

#### **Initial State** | Starting Client:
The starting point of the ransomware infection. This is where the kill chain attack originates from.

In [ ]:
starting_host.show()
starting_host.file_system.show()

In [ ]:
starting_host.software_manager.show()

#### **Initial State** | Database Server:

TAP001 opts to attack the UC7 database server (`ST_DATA-PRV-SRV-DB`):

In [ ]:
database_server.software_manager.show()
database_server.file_system.show()
database_server.show()

## **Kill Chain** | Kill Chain Stage Demonstration

### **Kill Chain** | NOT STARTED

This stage indicates that the TAP001 hasn't begun it's kill chain. 
TAP001 will begin the Mobile Malware Kill chain on the next execution step.

In [ ]:
env.reset();
tap001 = env.game.agents.get("attacker")
# The TAP001's Starting Client:
starting_host = env.game.simulation.network.get_node_by_hostname(tap001.starting_node)
# The TAP001's Database Server:
target_host = env.game.simulation.network.get_node_by_hostname('ST_DATA-PRV-SRV-DB')

In [ ]:
print(f"Current Environment Step: {env.game.step_counter}")
print(f"Current Kill Chain Stage: {tap001.current_kill_chain_stage.name}")
print(f"Next Execution Step: {tap001.next_execution_timestep}")
print(f"Next Kill Chain Stage {tap001.next_kill_chain_stage.name}")
while(tap001.current_kill_chain_stage == MobileMalwareKillChain.NOT_STARTED):
    default_obs, _, _, _, _ = env.step(0)

### **Kill Chain** | DOWNLOAD

|Index|Action Stage|OBS Impact|Narrative|
|-----|------------|----------|---------|
|1|DOWNLOAD|HOST:FOLDER:FILE:*|The `some_tech` employee encounters a malicious payload through web browsing on his mobile phone, often disguised as legitimate content.|

<p align="center">
    <a href="./_package_data/uc7/uc7_tap001/uc7_tap001_download_install.png" target="_blank">
        <img src="./_package_data/uc7/uc7_tap001/uc7_tap001_download_install.png" alt="Image" style="width:700px;">
    </a>  
</p>

In this stage, TAP001 uses the **node-folder-create** and **node-file-create** to create a file called ```"malware_dropper.ps1"``` within a ```"Downloads"``` folder. <br>
These actions are intended to simulate the malicious payload creating a  ```ps1``` (A windows powershell script) malware dropper on the `SOME_TECH` employee's phone. 

Currently, PrimAITE cannot simulate hosts joining the simulation mid-episode thus we must treat `ST_PROJ-A-PRV-PC-1`'s as also including the employee's phone. <br>
From a narrative perspective, this could be explained as the employee plugging his phone into the `ST_PROJ-A-PRV-PC-1`.

Additionally, it's worth noting that in the real world, malware droppers (small scripts or executables which download/install the malware after initially entering a host) use a variety of obfuscation methods to avoid detection. <br> For example, some malware droppers are concealed within legitimate files such as word document macros in order to trick a user into running the dropper.

Currently, PrimAITE's simulation does not operate a high enough fidelity to faithfully represent these techniques and their impacts to warrant their inclusion. Thus the file has been named as ```"malware_dropper"``` to prevent any potential confusion.

In [ ]:
print(f"Current Environment Step: {env.game.step_counter}")
print(f"Current Kill Chain Stage: {tap001.current_kill_chain_stage.name}")
print(f"Next Execution Step: {tap001.next_execution_timestep}")
print(f"Next Kill Chain Stage {tap001.next_kill_chain_stage.name}")
while(tap001.current_kill_chain_stage == MobileMalwareKillChain.DOWNLOAD):
    download_obs_impact, _, _, _, _ = env.step(0)
starting_host.file_system.show(full=True)
starting_host.file_system.show_num_files()
display_obs_diffs(default_obs, download_obs_impact, env.game.step_counter)

In [ ]:
tap001.logger.show()
tap001.show_history()

### **Kill Chain** | INSTALL
|Index|Attack Stage|OBS Impact|Narrative|
|-----|------------|----------|---------|
|2|Install | HOST:FOLDER:FILES:num_access |The malware is activated, either by the employee opening the file or automatically upon download, initiating its malicious functions.|

<p align="center">
    <a href="./_package_data/uc7/uc7_tap001/uc7_tap001_download_install.png" target="_blank">
        <img src="./_package_data/uc7/uc7_tap001/uc7_tap001_download_install.png" alt="Image" style="width:700px;">
    </a>  
    
</p>

In this stage, TAP001 uses the **node-file-access** to increase the number of accesses of the ```"malware_dropper.ps1"```. <br>

These actions represent the employee executing malware dropper created in the previous stage. 

In [ ]:
print(f"Current Environment Step: {env.game.step_counter}")
print(f"Current Kill Chain Stage: {tap001.current_kill_chain_stage.name}")
print(f"Next Execution Step: {tap001.next_execution_timestep}")
print(f"Next Kill Chain Stage {tap001.next_kill_chain_stage.name}")
while(tap001.current_kill_chain_stage == MobileMalwareKillChain.INSTALL):
    install_obs_impact, _, _, _, _ = env.step(0)

malware_file = starting_host.file_system.get_file("downloads","malware_dropper.ps1")
malware_file.show()

display_obs_diffs(download_obs_impact, install_obs_impact, env.game.step_counter)

In [ ]:
tap001.logger.show()
tap001.show_history()

### **Kill Chain** | ACTIVATE

|Index|Attack Stage|OBS Impact|Narrative|
|-----|------------|----------|---------|
|3|Activate | HOST:APPLICATION:operating_status |The malware installs itself on the terminal, often seeking to gain persistence by embedding itself into system processes or startup routines.|

<p align="center">
    <a href="./_package_data/uc7/uc7_tap001/uc7_tap001_activate.png" target="_blank">
        <img src="./_package_data/uc7/uc7_tap001/uc7_tap001_activate.png" alt="Image" style="width:700px;">
    </a>  
    
</p>

In this stage, TAP001 uses the **node-application-install** to install the ransomware application onto the starting host. <br>

These actions represent the malware dropper successfully installing ransomware on the host machine. Similarly to the malware dropper, the ransomware currently implemented is intended to be a generic and OS agnostic ransomware which is not intended to represent any specific real world implementation. 

Future versions of PrimAITE intend to expand the capability of the ransomware application to more faithfully represent a real-world example; for example, a Trickbot variation such as Ryuk or Conti.

In [ ]:
print(f"Current Environment Step: {env.game.step_counter}")
print(f"Current Kill Chain Stage: {tap001.current_kill_chain_stage.name}")
print(f"Next Execution Step: {tap001.next_execution_timestep}")
print(f"Next Kill Chain Stage {tap001.next_kill_chain_stage.name}")
while(tap001.current_kill_chain_stage == MobileMalwareKillChain.ACTIVATE):
    activate_obs_impact, _, _, _, _ = env.step(0)
starting_host.software_manager.show()
display_obs_diffs(install_obs_impact, activate_obs_impact, env.game.step_counter)

In [ ]:
tap001.logger.show()

### **Kill Chain** | PROPAGATE

|Index|Attack Stage|OBS Impact|Narrative|
|-----|------------|----------|---------|
|4|Propagate|HOST:NICS:NIC:TRAFFIC:PROTOCOL:PORT:*|The malware attempts to spread to other systems or networks, looking for vulnerable services.|

In this stage, TAP001 uses **node-nmap-port-scan**, **node-nmap-ping-scan** and **node-nmap-network-service-recon** to scan the simulation in order to search for a valid database target.

Unlike previous stages, the behaviour of this stage is dependant on the simulation and thus will perform differently dependant on the location of the target as well as the topology of the network. Specifically, the ```PROPAGATE``` stage uses three network enumeration actions and their action responses to populate its knowledge of the network.<br>
These actions represent the now infected `ST_PROJ-A-PRV-PC-1` searching the UC7 network for valid targets ransomware. 

For more information around how agent requests and responses work then the [request-response notebook can provide some useful insights](./Requests-and-Responses.ipynb).

It's worth noting that the implementation of the ```PROPAGATE``` stage does not class routers and switches as valid targets and thus will not scan them. <br>

_Currently, red agents have no way of probing the routing information of the simulation so must be provided the network addresses of the network. <br> See the mobile malware configuration option section of this notebook for further information_

#### **Kill Chain** | PROPAGATE | Scan walkthrough

The next juypter cells of this notebook will go through each individual nmap action that the TAP001 leverages to reach the target host as well as the OBS each action impacts.

This section uses the following ```PROPAGATE``` relevant TAP001 settings:
```yaml
    kill chain:
        PROPAGATE:
          probability: 1
          scan_attempts: 20
          repeat_scan: false
          network_addresses:
            - 192.168.230.0/29 # ST Project A
            - 192.168.10.0/26  # Remote Site
            - 192.168.20.0/30  # Remote DMZ
            # - 192.168.240.0/29 # ST Project B
            # - 192.168.250.0/29 # ST Project C
            - 192.168.220.0/29 # ST Data (Contains Target)
```


In [ ]:
print(f"Current Environment Step: {env.game.step_counter}")
print(f"Current Kill Chain Stage: {tap001.current_kill_chain_stage.name}")
print(f"Next Execution Step: {tap001.next_execution_timestep}")
print(f"Next Kill Chain Stage {tap001.next_kill_chain_stage.name}")

def propagate_obs_and_show(tap001: TAP001, previous_obs, propagate_obs_impact, timestep):
    tap001._show_scan()
    display_obs_diffs(previous_obs, propagate_obs_impact, timestep)

In [ ]:
tap001.logger.show()

#### **Kill Chain** | PROPAGATE | ST_PROJ-A-PRV-PC-1

<p align="center">
    <a href="./_package_data/uc7/uc7_tap001/uc7_tap001_propagate_1.png" target="_blank">
        <img src="./_package_data/uc7/uc7_tap001/uc7_tap001_propagate_1.png" alt="Image" style="width:700px;">
    </a>  
    
</p>

In [ ]:
# Ping Scan
for _ in range(5):
    PROJ_A_SITE_ping, *_ = env.step(0)

In [ ]:
propagate_obs_and_show(tap001, previous_obs=activate_obs_impact, propagate_obs_impact=PROJ_A_SITE_ping, timestep=env.game.step_counter)

In [ ]:
# Recon Scan
for _ in range(5):
    PROJ_A_SITE_recon, *_  = env.step(0);

In [ ]:
propagate_obs_and_show(tap001, PROJ_A_SITE_ping, PROJ_A_SITE_recon, env.game.step_counter)

#### **Kill Chain** | PROPAGATE | REMOTE SITE

<p align="center">
    <a href="./_package_data/uc7/uc7_tap001/uc7_tap001_propagate_2.png" target="_blank">
        <img src="./_package_data/uc7/uc7_tap001/uc7_tap001_propagate_2.png" alt="Image" style="width:1000px;">
    </a>  
    
</p>

In [ ]:
# Ping Scan
for _ in range(5):
    REMOTE_SITE_ping, *_ = env.step(0)

In [ ]:
propagate_obs_and_show(tap001, previous_obs=activate_obs_impact, propagate_obs_impact=REMOTE_SITE_ping, timestep=env.game.step_counter)

In [ ]:
# Recon Scan
for _ in range(5):
    REMOTE_SITE_recon, *_  = env.step(0);

In [ ]:
propagate_obs_and_show(tap001, REMOTE_SITE_ping, REMOTE_SITE_recon, env.game.step_counter)

#### **Kill Chain** | PROPAGATE | REMOTE DMZ

<p align="center">
    <a href="./_package_data/uc7/uc7_tap001/uc7_tap001_propagate_3.png" target="_blank">
        <img src="./_package_data/uc7/uc7_tap001/uc7_tap001_propagate_3.png" alt="Image" style="width:700px;">
    </a>  
    
</p>

In [ ]:
# Ping Scan
for _ in range(5):
    REMOTE_DMZ_ping, *_ = env.step(0)

In [ ]:
propagate_obs_and_show(tap001, previous_obs=REMOTE_SITE_recon, propagate_obs_impact=REMOTE_DMZ_ping, timestep=env.game.step_counter)

In [ ]:
# Recon Scan
for _ in range(5):
    REMOTE_DMZ_recon, *_  = env.step(0);

In [ ]:
propagate_obs_and_show(tap001, previous_obs=REMOTE_DMZ_ping, propagate_obs_impact=REMOTE_DMZ_recon, timestep=env.game.step_counter)

#### **Kill Chain** | PROPAGATE | SOME_TECH DATA 


<p align="center">
    <a href="./_package_data/uc7/uc7_tap001/uc7_tap001_propagate_4.png" target="_blank">
        <img src="./_package_data/uc7/uc7_tap001/uc7_tap001_propagate_4.png" alt="Image" style="width:700px;">
    </a>  
    
</p>

In [ ]:
# Ping Scan
for _ in range(5):
    ST_DATA_ping, *_ = env.step(0)

In [ ]:
propagate_obs_and_show(tap001, previous_obs=REMOTE_DMZ_recon, propagate_obs_impact=ST_DATA_ping, timestep=env.game.step_counter)

In [ ]:
# Recon Scan
for _ in range(5):
    ST_DATA_recon, *_ = env.step(0)

In [ ]:
propagate_obs_and_show(tap001, previous_obs=ST_DATA_ping, propagate_obs_impact=ST_DATA_recon, timestep=env.game.step_counter)

In [ ]:
for _ in range(5):
    propagate_obs_impact, *_  = env.step(0)

In [ ]:
tap001._show_scan()

In [ ]:
tap001.logger.show()

### **Kill Chain** | COMMAND_AND_CONTROL
|Index|Attack Stage|OBS Impact|Narrative|
|-----|------------|----------|---------|
|5|Command and Control| **Starting Host** HOST:NICS:NIC:TRAFFIC:PROTOCOL:PORT:* HOST:APPLICATIONS:APPLICATION:* |The malware establishes a connection to an external command and control (C&C) server, receiving instructions and possibly exfiltrating data.|


<p align="center">
    <a href="./_package_data/uc7/uc7_tap001/uc7_tap001_c2.png" target="_blank">
        <img src="./_package_data/uc7/uc7_tap001/uc7_tap001_c2.png" alt="Image" style="width:700px;">
    </a>  
    
</p>


For further details please refer to the [Command-and-Control-E2E-Demonstration notebook](./Command-and-Control-E2E-Demonstration.ipynb).

_Note: The referenced notebook above uses the UC2 scenario for demonstration purposes, however all the OBS impacts and C2 suite functionality is equally applicable to UC7._



In [ ]:
print(f"Current Environment Step: {env.game.step_counter}")
print(f"Current Kill Chain Stage: {tap001.current_kill_chain_stage.name}")
print(f"Next Execution Step: {tap001.next_execution_timestep}")
print(f"Next Kill Chain Stage {tap001.next_kill_chain_stage.name}")
while(tap001.current_kill_chain_stage == MobileMalwareKillChain.COMMAND_AND_CONTROL):
    c_and_c_obs_impact, _, _, _, _ = env.step(0)

In [ ]:
display_obs_diffs(propagate_obs_impact, c_and_c_obs_impact,env.game.step_counter)

In [ ]:
tap001.logger.show()

### **Kill Chain** | PAYLOAD - Corruption & Exfiltration
|Index|Attack Stage|OBS Impact|Narrative|
|-----|------------|----------|---------|
|6A|Corruption| **Target Host** HOST:FOLDERS:FOLDER:FILES:FILE:num_access HOST:num_file_creations  HOST:num_file_deletions  HOST:FOLDERS:FOLDER:FILES:FILE:health_status | The attacker configures the ransomware script to target the IP address of the discovered database, then it performs it's intended malicious activities, allowing it to corrupt the Database whilst leaving the database service operable|
|6B|Exfiltration| **Starting Host** HOST:FOLDER:FILES:FILE HOST:NICS:NIC:TRAFFIC:PROTOCOL:PORT:* HOST:APPLICATIONS:APPLICATION:* | The attacker remotely logins into the target host and exfiltrates the database.db file onto the starting host which then relays this file back to the C2 Server.|


<p align="center">
    <a href="./_package_data/uc7/uc7_tap001/uc7_tap001_payload.png" target="_blank">
        <img src="./_package_data/uc7/uc7_tap001/uc7_tap001_payload.png" alt="Image" style="width:700px;">
    </a>  
    
</p>


In [ ]:
print(f"Current Environment Step: {env.game.step_counter}")
print(f"Current Kill Chain Stage: {tap001.current_kill_chain_stage.name}")
print(f"Next Execution Step: {tap001.next_execution_timestep}")
print(f"Next Kill Chain Stage {tap001.next_kill_chain_stage.name}")
while(tap001.current_kill_chain_stage == MobileMalwareKillChain.PAYLOAD):
    payload_obs_impact, _, _, _, _ = env.step(0)

In [ ]:
target_host.file_system.show(full=True)
display_obs_diffs(c_and_c_obs_impact, payload_obs_impact, env.game.step_counter)

In [ ]:
tap001.logger.show()

We can use the blue agent's NODE_FILE_SCAN action to scan the database file:

```yaml
# ST_DATA-PRV-SRV-DB | node-file-scan | Scans the database.db file (health status)
49:
    action: node-file-scan
    options:
        node_name: ST_DATA-PRV-SRV-DB
        folder_name: database
        file_name: database.db
```

*You should notice a file `health_status` change to a value of 3*

In [ ]:
env.step(0)
post_scan_payload_impact, _, _, _, _ = env.step(49) # 
display_obs_diffs(payload_obs_impact, post_scan_payload_impact, env.game.step_counter)

In [ ]:
target_host.file_system.show(full=True)

We can also see that the `database.db` file was successfully exfiltrated.

In [ ]:
c2_server = env.game.simulation.network.get_node_by_hostname(tap001.c2_settings['c2_server'])
starting_host.file_system.show(full=True)
c2_server.file_system.show(full=True)

In [ ]:
tap001.logger.show()

### **Attack Configurations** 

All TAP's inherit the same general configurability shown in the following table:

<details> <summary> TAP Agent Config Options </summary>

|Option Field|Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|start_step|The initial kill chain starting step.|Int|_Required_|
|frequency|The frequency of each kill chains starting steps.|Int|_Required_|
|variance| The timestep variance between frequency|Int|_Required_|
|repeat_kill_chain|Indicates whether the attack is repeated throughout the episode.|Bool|_Required_|
|repeat_kill_chain_stages|Indicates if the kill_chain stage should reset upon failure or retry.|Bool|_Required_|
|default_target_ip | The IP address of the target host |Str|_Required_|
|default_target_node|The Target Host|Str|_Required_|
|target_nodes|A list of Potential Target Hosts (database services) - Selected on per episode basis.|List|_Optional_|
|default_starting_node|The Starting Host|Str|_Required_|
|starting_nodes|A list of Potential Targets|List|_Optional_|
|kill_chain|TAP001 Specific Config (_See the next notebook section_)|Dict|_Required_|
|description|Free Text|Str|_Optional_|

</details>

```yaml

  - ref: attacker
    team: RED
    type: TAP001
    agent_settings:
      start_step: 1
      frequency: 3
      variance: 0
      repeat_kill_chain: false
      repeat_kill_chain_stages: true
      default_target_ip: *ST_SRV_DB_IP
      default_starting_node: "ST_PROJ-A-PRV-PC-1"
      # starting_nodes: ["ST_PROJ-A-PRV-PC-1", "ST_PROJ-B-PRV-PC-2", "ST_PROJ-C-PRV-PC-3"]
      kill_chain:
        ... # Next notebook section will cover this configuration option

```

This notebook assumes the reader is already familiar with the below `agent_settings` and won't provide any code snippet examples. 

|Option Field|Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|start_step|The initial kill chain starting step.|Int|_Required_|
|frequency|The frequency of each kill chains starting steps.|Int|_Required_|
|variance| The timestep variance between frequency|Int|_Required_|
|repeat_kill_chain|Indicates whether the attack is repeated throughout the episode.|Bool|_Required_|
|repeat_kill_chain_stages|Indicates if the kill_chain stage should reset upon failure or retry.|Bool|_Required_|
|default_target_node|The Target Host|Str|_Required_|
|starting_nodes|A list of Potential Targets|List|_Optional_|
|kill_chain|_See the next notebook section_|Dict|_Required_||


If you're unfamiliar with the above then refer to the equivalent `agent_settings` section within the [TAP003 Kill Chain E2E Demonstration notebook](./TAP003-Kill-Chain-E2E.ipynb). 

In [ ]:
tap001 = env.game.agents.get("attacker")
for key,value in tap001.config.agent_settings:
    if key == 'kill_chain':
        pass
    else:
        print(f"{key} : {value}")

### **Attack Configurations** | Kill Chain Settings

Additionally, TAP001's Mobile Malware Kill Chain comes with some extra configuration options. 

These options can be configured to customise the behaviour of certain stages. 

The YAML snippet below is the current default configuration of the mobile malware kill chain:

``` YAML
  - ref: attacker
    team: RED
    type: TAP001
    agent_settings:
      kill_chain:
        ACTIVATE:
          probability: 1
        PROPAGATE:
          probability: 1
          scan_attempts: 20
          repeat_scan: false
          network_addresses:
            - 192.168.230.0/29 # ST Project A
            - 192.168.10.0/26  # Remote Site
            - 192.168.20.0/30  # Remote DMZ
            # - 192.168.240.0/29 # ST Project B
            # - 192.168.250.0/29 # ST Project C
            - 192.168.220.0/29 # ST Data (Contains Target)
        COMMAND_AND_CONTROL:
          probability: 1
          keep_alive_frequency: 5
          masquerade_port: HTTP
          masquerade_protocol: TCP
          c2_server_name: ISP-PUB-SRV-DNS
          c2_server_ip: *PUBLIC_DNS_IP
        PAYLOAD:
          probability: 1
          exfiltrate: true
          corrupt: true
          exfiltration_folder_name:
          target_username: admin
          target_password: admin
          continue_on_failed_exfil: True
```

#### **Attack Configurations** | Mobile Malware Kill Chain | General Settings

TAP001's ``ACTIVATE``, ``PROPAGATE``, ``COMMAND_AND_CONTROL`` and ``PAYLOAD`` stages's probability of success can be configured.

Similarly to other TAPs, the argument given to probability is the chance of action performing that stage of kill chain successfully. 
The argument given is expected to be between **0** - **1**.  

_(With '1' equalling 100% chance of 'success')_

It's important to note that the probabilities of success are calculated within the game layer meaning that if an TAP fails because of probability then the TAP will perform a ``DONOTHING`` action for that step. 

```yaml
        kill_chain:
          ACTIVATE:
            probability: 1
          PROPAGATE:
            probability: 1
          COMMAND_AND_CONTROL:
            probability: 1
          PAYLOAD:
            probability: 1
```

#### **Attack Configurations** | Mobile Malware Kill Chain | Propagate Stage

TAP001's propagate step leverages the NMAP application to scan the network. This kill chain stage is a considerably more complex than the other stages and thus has more configuration options.

<details> <summary> Propagate Configuration Settings </summary>

|Option Field |Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|probability|Action Probability - This is only calculated on the initial scan.|int|_Required_|
|network_addresses|The network-addresses that are scanned during the propagate step. Scanned in the order they are defined.|list[str]|_Required_|
|scan_attempts|The amount of permitted scan attempts.|int|_Required_|
|repeat_scan|Should the scan repeat if the target is not found within the given network address|bool|_optional_|

</details> 

**Probability**

***

|Option Field |Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|probability|Action Probability |Int|_Required_|

Similarly, to every other stage, probability is the chance of success of TAP001 has to successfully perform the Propagate Stage.

However, it's important to note that the **probability of success is only calculated once**. 

After the first scan is performed, the TAP agent will perform the rest of the stage without trialing probability.

**Network Addresses**

***

|Option Field |Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|network_addresses|The network-addresses that are scanned during the propagate step. Scanned in the order they are defined.|list[str]|_Required_|

At present, TAPs cannot probe the simulation for routing information.

Therefore, to scan multiple networks, the ``PROPAGATE`` stage must be provided a list of each network address.

These network's are scanned in sequential order.

For example; this notebook is configured with the following ``PROPAGATE`` network_address setting.

| Network Address | Use Case 7 Subnet Name  |
|-----------------|-------------------------|
| 192.168.10.0/26  | REMOTE-SITE            |
| 192.168.20.0/30  | REMOTE-DMZ             |
| 192.168.220.0/29 | SOME_TECH_DATA         |


Which when represented in the yaml config is as follows:

```yaml
      kill_chain:
        PROPAGATE:
          probability: 1
          scan_attempts: 20
          repeat_scan: false
          network_addresses:
            - 192.168.230.0/29 # ST Project A
            - 192.168.10.0/26  # Remote Site
            - 192.168.20.0/30  # Remote DMZ
            # - 192.168.240.0/29 # ST Project B
            # - 192.168.250.0/29 # ST Project C
            - 192.168.220.0/29 # ST Data (Contains Target)
```
Which is loaded into the TAP001 agent as the following list:
```Python
["192.168.230.0/29", "192.168.10.0/26", "192.168.20.0/30", "192.168.220.0/29"]
```

As PrimAITE expands and TAP agents are provided more ways of probing the simulation, then the ```PROPAGATE``` stage will be able to perform independently thus no longer requiring the network_address configuration option. 

**Scan Attempts**

***
|Option Field |Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|scan_attempts|The amount of permitted scan attempts.|int|_Required_|

Simply, the ``scan_attempts`` configuration option indicates how many SCAN actions ``PROPAGATE`` stage is permitted to perform before the kill chain is considered to have failed. 

The ``scan_attempts`` option should mainly be kept higher than the length of the list provided in the previous ``network_address`` option.

Currently, this setting is mainly used in conjunction with next setting: ```repeat_scan```.


**Repeat Scan**

***
|Option Field |Meaning|Expected Type|Required/Optional|
|------------|-------|-------------|-----------------|
|repeat_scan|Should the scan repeat if the target is not found within the given network address|bool|_optional_|

This boolean flag controls whether the ```PROPAGATE``` stage should repeat if the target is not found within the given ```network_addresses```.

This setting used in conjunction with the ```permitted_scan``` option allows the Red Agent to continue scanning even if the target is not found. 

The network addresses will be selected at random after the first sequential scan(s) of the given ```network_addresses``` for further domain randomisation. 

In [ ]:
# Removing the target network address. (Thus the target will never be found)
network_addresses = ["192.168.10.0/26", "192.168.20.0/30"]

with open(_EXAMPLE_CFG/"uc7_config.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg['io_settings']['save_sys_logs'] = True # Saving syslogs
    cfg['agents'][32]['agent_settings']['repeat_kill_chain_stages'] = True
    cfg['agents'][32]['agent_settings']['kill_chain']['PROPAGATE']['network_addresses'] = network_addresses
    cfg['agents'][32]['agent_settings']['kill_chain']['PROPAGATE']['probability'] = 1
    cfg['agents'][32]['agent_settings']['kill_chain']['PROPAGATE']['scan_attempts'] = 30
    cfg['agents'][32]['agent_settings']['kill_chain']['PROPAGATE']['repeat_scan'] = True
env = PrimaiteGymEnv(env_config = cfg)
env.reset()
for _ in range (256):
    env.step(0)

***

<sup>[1]</sup> _PrimAITE does not actually enforce agent type (Red/Green/Blue) specific actions_

_However, some actions such as `node-application-execute` and `node-nmap-network-service-recon` require an understanding of the simulation that is beyond the blue agent's current observation and thus are not suitable for use by reinforcement algorithms._

_These actions are usually only leveraged by Green or Red agents; hence why they are commonly referenced as such._

***

#### **Attack Configurations** | Mobile Malware Kill Chain | Command and Control Stage

TAP001's Command and Control stage leverages the `c2-beacon` which has it's own set of configuration options. In the case of TAP001 some of these settings are already pre-defined based on other settings such as ``target_node``. The table below details the currently available options.

<details> <summary> Command and Control Configuration Settings </summary>

|Option Field         | Meaning                                                                            |Expected Type  |Required/Optional|
|---------------------|------------------------------------------------------------------------------------|---------------|-----------------|
|probability          | Action Probability - This is only calculated once at this stage.                   |Int            | _Required_      |
|c2_server            | What host should the C2 Beacon attempt to connect to as the chosen C2 Server       |Str            | _Required_      |
|keep_alive_frequency | How often should the C2 Beacon confirm its connection in timesteps. Defaults to 5  |Int            | _Optional_      |
|masquerade_port      | What port should the C2 traffic use? Defaults to TCP.                              |Str            | _Optional_      |
|masquerade_protocol  | What protocol should the C2 traffic masquerade as? Defaults to HTTP.               |Str            | _Optional_      |

</details> 

For further information around the configuration of the `c2-beacon` please refer to the [Command-&-Control-E2E-Demonstration notebook](./Command-and-Control-E2E-Demonstration.ipynb)'s last section on configurability.

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    # Adding the C2 Server to a different node (REM-PUB-PC-1)
    cfg['simulation']['network']['nodes'][10].update(
        {"applications": [
            {"type": "database-client", "options":{"db_server_ip":"192.168.220.3"}},
            {"type": "web-browser", "options":{"target_url": "http://some_tech.com"}},
            {"type": "c2-server", "options":{"listen_on_ports":[21]}}]
        })

    # Configuring the C2 stage to use the REM-PUB-PC-1 as it's C2 Server and to use a different masquerade port.
    cfg['agents'][32]['agent_settings']['kill_chain']['COMMAND_AND_CONTROL']["c2_server_name"] = "REM-PUB-PC-1"
    cfg['agents'][32]['agent_settings']['kill_chain']['COMMAND_AND_CONTROL']["c2_server_ip"] = "192.168.20.2"
    cfg['agents'][32]['agent_settings']['kill_chain']['COMMAND_AND_CONTROL']["keep_alive_frequency"] = 3
    cfg['agents'][32]['agent_settings']['kill_chain']['COMMAND_AND_CONTROL']["masquerade_port"] = "FTP"
    cfg['agents'][32]['agent_settings']['kill_chain']['COMMAND_AND_CONTROL']["masquerade_protocol"] = "TCP"
env = PrimaiteGymEnv(env_config = cfg)
env.reset()
# TAP001 requires around 110 timesteps using default TAP settings.
for _ in range(110):
    env.step(0)

The code cells below use .show() methods to show that the configuration options have successfully altered the C2's suite configuration. For example the `c2-beacon`'s remote connection is now ``REM-PUB-PC-1``'s ip address which is ``192.168.20.2``.

In [ ]:
starting_host = env.game.simulation.network.get_node_by_hostname('ST_PROJ-A-PRV-PC-1')
c2_beacon = starting_host.software_manager.software["c2-beacon"]
c2_beacon.show()

In [ ]:
c2_server_host = env.game.simulation.network.get_node_by_hostname('REM-PUB-PC-1')
c2_server = c2_server_host.software_manager.software["c2-server"]
c2_server.show()

#### **Attack Configurations** | Mobile Malware Kill Chain | Payload Stage

<details> <summary> Payload Settings </summary>

|Option Field         | Meaning                                                                            |Expected Type  |Required/Optional|
|---------------------|------------------------------------------------------------------------------------|---------------|-----------------|
|probability          | Action Probability - This is only calculated once at this stage.                   |Int            | _Required_      |
|corrupt              | Should TAP001 launch the ransomware script against the target database?            |Boolean        | _Required_      |
|exfiltrate           | Should TAP001 exfiltrate the target database.db file?                              |Boolean        | _Required_      |

</details>

**Probability**

***

|Option Field         | Meaning                                                                            |Expected Type  |Required/Optional|
|---------------------|------------------------------------------------------------------------------------|---------------|-----------------|
|probability          | Action Probability - This is only calculated once at this stage.                   |Int            | _Required_      |

Similarly, to every other stage, probability is the chance of success of TAP001 has to successfully perform the Payload Stage.

However, it's important to note that the **probability of success is only calculated once**. 

After the agent is successful once then the TAP agent will perform the rest of the stage without trialing any further.

**Corrupt**

***

|Option Field         | Meaning                                                                            |Expected Type  |Required/Optional|
|---------------------|------------------------------------------------------------------------------------|---------------|-----------------|
|corrupt              | Should TAP001 launch the ransomware script against the target database?            |Boolean        | _Required_      |

This option is a boolean value which indicates if TAP001 should launch the ransomware script against the target database. 

By default this is enabled but if users wish to disable the ransomware attack for training purposes then this value can be set to ``False`` which will prevent the installed ``RansomwareScript`` from launching at the final step.



**Exfiltrate**

***

|Option Field         | Meaning                                                                            |Expected Type  |Required/Optional|
|---------------------|------------------------------------------------------------------------------------|---------------|-----------------|
|exfiltrate           | Should TAP001 exfiltrate the target database.db file?                              |Boolean        | _Required_      |


Similar to ``corrupt``, this option is a boolean value which indicates if TAP001 should attempt to exfiltrate the `database.db` file.

By default this is enabled but if users wish to disable the exfiltration for training purposes then this value can be set to ``False`` which will prevent the TAP001 agent from attempting to exfiltrate the database.db file.

_If both ``exfiltrate`` and ``corrupt`` options are enabled then the TAP001 agent will exfiltrate the database.db and then launch the ``ransomware-script`` against the target._

_yaml config example_

```yaml
      kill_chain:
        PAYLOAD:
          probability: 1
          exfiltrate: true
          corrupt: true
          exfiltration_folder_name:
          target_username: admin
          target_password: admin
          continue_on_failed_exfil: True
```

The following code cells demonstrate how ``corrupt`` and ``exfiltrate`` can be enabled/disabled and their effects on the environment. After each example the file systems of the target and starting hosts are displayed. If ``corrupt`` is enabled then the target database file will be encrypted and if the ``exfiltrate`` option is enabled then the target database.db will be present within the starting host's file system.

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg['agents'][32]['agent_settings']['kill_chain']['PAYLOAD']["corrupt"] = True
    cfg['agents'][32]['agent_settings']['kill_chain']['PAYLOAD']["exfiltrate"] = False
env = PrimaiteGymEnv(env_config = cfg)
env.reset()
for _ in range(110):
    env.step(0)

In [ ]:
starting_host = env.game.simulation.network.get_node_by_hostname('ST_PROJ-A-PRV-PC-1')
target_host = env.game.simulation.network.get_node_by_hostname('ST_DATA-PRV-SRV-DB')
target_host.file_system.show(full=True)
starting_host.file_system.show(full=True)

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg['agents'][32]['agent_settings']['kill_chain']['PAYLOAD']["corrupt"] = False
    cfg['agents'][32]['agent_settings']['kill_chain']['PAYLOAD']["exfiltrate"] = True
env = PrimaiteGymEnv(env_config = cfg)
env.reset()
for _ in range(110):
    env.step(0)


In [ ]:
starting_host = env.game.simulation.network.get_node_by_hostname('ST_PROJ-A-PRV-PC-1')
target_host = env.game.simulation.network.get_node_by_hostname('ST_DATA-PRV-SRV-DB')
target_host.file_system.show(full=True)
starting_host.file_system.show(full=True)

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg['agents'][32]['agent_settings']['kill_chain']['PAYLOAD']["corrupt"] = True
    cfg['agents'][32]['agent_settings']['kill_chain']['PAYLOAD']["exfiltrate"] = True
env = PrimaiteGymEnv(env_config = cfg)
env.reset()
for _ in range(110):
    env.step(0)

In [ ]:
starting_host = env.game.simulation.network.get_node_by_hostname('ST_PROJ-A-PRV-PC-1')
target_host = env.game.simulation.network.get_node_by_hostname('ST_DATA-PRV-SRV-DB')
target_host.file_system.show(full=True)
starting_host.file_system.show(full=True)

#### Payload Stage | Exfiltration Settings

If TAP001 is configured to exfiltrate the database.db file then the following configuration options are relevant.

<details> <summary> Exfiltration Settings </summary>

|Option Field             | Meaning                                                                            |Expected Type  |Required/Optional|
|-------------------------|------------------------------------------------------------------------------------|---------------|-----------------|
|exfiltration_folder_name | The folder used to store the database.db file. Defaults to ``exfiltration_folder`` |Str            | _Optional_      |
|target_username          | The username used to login into a target node. Defaults to ``admin``               |Str            | _Required_      |
|target_password          | The password used to login into a target node. Defaults to ``admin``               |Str            | _Required_      |
|continue_on_failed_exfil | Indicates if the attacker should encrypt the target even if the exfiltration fails |Bool           | _Required_      |

</details> 

**exfiltration_folder_name**

***

|Option Field             | Meaning                                                                            |Expected Type  |Required/Optional|
|-------------------------|------------------------------------------------------------------------------------|---------------|-----------------|
|exfiltration_folder_name | The folder used to store the database.db file. Defaults to ``exfiltration_folder``.|str            | _Optional_      |

Users are able to configure what folder the database.db is stored within after successful database exfiltration. For example, if this option was set to ``crown_jewels`` then the TAP001 agent would create a new folder on the starting node called ``crown_jewels`` and once the data-exfiltration is successful that folder will be populated with the ``database.db`` file from the target database. 

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg['agents'][32]['agent_settings']['kill_chain']['PAYLOAD']["exfiltration_folder_name"] = "crown_jewels"
env = PrimaiteGymEnv(env_config = cfg)
env.reset()
for _ in range(110):
    env.step(0)

In [ ]:
starting_host = env.game.simulation.network.get_node_by_hostname('ST_PROJ-A-PRV-PC-1')
target_host = env.game.simulation.network.get_node_by_hostname('ST_DATA-PRV-SRV-DB')
target_host.file_system.show(full=True)
starting_host.file_system.show(full=True)

**target_username & target_password**

***

|Option Field             | Meaning                                                                            |Expected Type  |Required/Optional|
|-------------------------|------------------------------------------------------------------------------------|---------------|-----------------|
|target_username          | The username used to login into a target node. Defaults to ``admin``               |Str            | _required_      |
|target_password          | The password used to login into a target node. Defaults to ``admin``               |Str            | _required_      |

These fields indicate what user credentials the TAP001 agent will use when attempting to exfiltrate the `database.db` file from the target.

In [ ]:
# Creating a new user for the TAP001 to use.
users = [{
    "username": "example_user",
    "password": "example_pass",
    "is_admin": "False",
}]
with open(_EXAMPLE_CFG/"uc7_config.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg['simulation']['network']['nodes'][28].update({"users":users}) # Adding this new user to the target
    cfg['agents'][32]['agent_settings']['kill_chain']['PAYLOAD']["target_username"] = "example_user" # Setting TAP001 to use the new user credentials.
    cfg['agents'][32]['agent_settings']['kill_chain']['PAYLOAD']["target_password"] = "example_pass"
env = PrimaiteGymEnv(env_config = cfg)
env.reset()
for _ in range(110):
    env.step(0)


In [ ]:
target_host = env.game.simulation.network.get_node_by_hostname('ST_DATA-PRV-SRV-DB')
target_host.user_manager.show()
target_host.file_system.show()
tap001.logger.show()

**continue_on_failed_exfil**

***
|Option Field             | Meaning                                                                            |Expected Type  |Required/Optional|
|-------------------------|------------------------------------------------------------------------------------|---------------|-----------------|
|continue_on_failed_exfil | Indicates if the attacker should encrypt the target even if the exfiltration fails |Bool           | _Required_      |


This option will affect how the TAP001 agent responds to failing the exfiltration. If this option is set to `True` then the TAP001 agent will attempt the final attack against the database even if the `database.db` exfiltration attempt is unsuccessful. Likewise, if this option is set to `False` then the TAP001 agent will consider it's attack failed if it cannot complete the exfiltration.

The code cells below demonstrate the differences by using the blue agent to change the user account details on the target database before the exfiltration occurs using action 50

```yaml
    # ST_DATA-PRV-SRV-DB | node-account-change-password | Changes the password of a user account
    50:
        action: node-account-change-password
        options:
        node_name: ST_DATA-PRV-SRV-DB
        username: admin   # default account
        current_password: admin   # default password
        new_password: thr33_alert_wolv3z # A more 'secure' password
```


If we set the `continue_on_failed_exfil` to true we can see that the despite the exfiltration failing the `database.db` still ends up corrupted by the end of the attack.

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg['agents'][32]['agent_settings']['kill_chain']['PAYLOAD']["continue_on_failed_exfil"] = True
    cfg['agents'][32]['agent_settings']['repeat_kill_chain'] = False
env = PrimaiteGymEnv(env_config = cfg)
env.reset()

# Changing the target database credentials:
env.step(50)

# Finishing the episode:
for _ in range(127):
    env.step(0)
    
tap001.logger.show()

In [ ]:
# The exfiltration was not successful
starting_host = env.game.simulation.network.get_node_by_hostname(tap001.starting_node)

starting_host.file_system.show()

In [ ]:
# Yet the target database.db is still corrupt.

target_host = env.game.simulation.network.get_node_by_hostname('ST_DATA-PRV-SRV-DB')
target_host.file_system.show()

Whereas now if we set `continue_on_failed_exfil` to false we can see that the attack failed and the database health status remains `GOOD`.

In [ ]:
with open(_EXAMPLE_CFG/"uc7_config.yaml", mode="r") as f:
    cfg = yaml.safe_load(f)
    cfg['agents'][32]['agent_settings']['kill_chain']['PAYLOAD']["continue_on_failed_exfil"] = False
    cfg['agents'][32]['agent_settings']['repeat_kill_chain'] = False
env = PrimaiteGymEnv(env_config = cfg)
env.reset()

# Changing the target database credentials:
env.step(50)

# Finishing the episode:
for _ in range(127):
    env.step(0)
    
tap001.logger.show()

In [ ]:
# The exfiltration was not successful
starting_host = env.game.simulation.network.get_node_by_hostname(tap001.starting_node)

starting_host.file_system.show()

In [ ]:
# And the target database.db remains healthy.

target_host = env.game.simulation.network.get_node_by_hostname('ST_DATA-PRV-SRV-DB')
target_host.file_system.show()